---

# <left> Introduction To Google Earth Engine </left>

---
---

## Objectives

Setup a Google Earth Engine account, gain familiarity with the Google Earth Engine (GEE) Code Editor using JavaScript and use the platform to build a geospatial application. The skills gained (hopefully!) can very quickly be expanded for more advanced geospatial work.  

1. Day 1 :
    - Setup a Google Earth Engine account, gain familiarity with the Code editor and Javascript for GEE.
    - Familiarise with the GEE Obects, Methods and Datasets.
2. Day 2 :
    - Use the skills from Day 1 to build a geospatial application. 

### Day Two

## Building a GeoSpatial application using GEE.

According to the GEE Official documentation, "Earth Engine Apps are dynamic, shareable user interfaces for Earth Engine analyses". GEE apps once published, are accesible as a standalone application with a unique URL. To view and interact with the apps requires no logging or sign up and is by ddefault open to public (any one with a browser can view and interact with your application). Most of the functions available within the GEE Code Editor is available for use while devoloping GEE apps, however as it uses the User Interface (UI) API, we will take a look at the GEE UI API. 

The GEE UI API consists of several components such as :

- Widgets
- Panels
- Events

Widgets are different user interfaces that are offered by GEE. These include buttons, dropdowns, sliders etc. Panels are containers or spaces, where the widgets are arranged. Each panel can hold several widgets with different arrangements, and a root panel provides the base to hold all other panels. Events are the action which are triggerd when the the user interacts with a widget.  

For those familiar with javascript will find that these are more or less JavaScript, albeit with some  adaptations for the GEE architecture. Let's look at the objects offered within GEE for each of this components. 

### Widgets and Events

Several widgets are available within GEE. Lets look at some of them and how to add them to the console. 

You could follow along by pasting the code. At this stage we will just print off to the console, later we will be placing the widgets inside panels.

First lets print a **label(ui.Label)**

~~~javascript
var label = ui.Label('My GEE Label!');
print(label);

// To break up a label to different lines :

var label = ui.Label('This app\ncalculates area \nas well as parameter', {whiteSpace: 'pre'});
print(label);
~~~

While a label just prints out a string, other widgets are more interactive like a **button (ui.Button)**. Lets add a button which will take us to a nice looking feature at Kolhumadulu Atoll.

~~~javascript
var button = ui.Button({
  label: 'Zoom to Kolhumadulu',
  onClick: function() {
    print(Map.setCenter(2.266393578083512, 73.22899822178088));
  }
});
print(button);
~~~

Now that we have a button, lets try a **checkbox (ui.Checkbox)**. Lets add a checkbox which loads a sentinel 2 image for the location we zoomed in earlier. 

~~~javascript
var checkbox = ui.Checkbox('Show Sentinel 2', true);

checkbox.onChange(function(checked) {
  // Shows or hides the first map layer based on the checkbox's value.
  Map.layers().get(0).setShown(checked);
});

function maskS2clouds(image) {
  var qa = image.select('QA60');

  // Bits 10 and 11 are clouds and cirrus, respectively.
  var cloudBitMask = 1 << 10;
  var cirrusBitMask = 1 << 11;

  // Both flags should be set to zero, indicating clear conditions.
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0)
      .and(qa.bitwiseAnd(cirrusBitMask).eq(0));

  return image.updateMask(mask).divide(10000);
}

var dataset = ee.ImageCollection('COPERNICUS/S2_SR')
                  .filterDate('2019-01-01', '2020-12-31')
                  // Pre-filter to get less cloudy granules.
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))
                  .map(maskS2clouds);

// visualisation parameters
var visualization = {
  min: 0.0,
  max: 0.3,
  bands: ['B4', 'B3', 'B2'],
};

// get the mean of the dataset
Map.setCenter(72.8935501941133, 2.6718531872448774,15);
Map.addLayer(dataset.mean(), visualization, 'RGB');
print(checkbox);
~~~

Several other widgets are available within the GEE platform. It is recommended to visit the [widgets](https://developers.google.com/earth-engine/guides/ui_widgets#ui.dateslider) GEE website to see the available widgets and examples.

An event is the resulting output due to user interaction. Observe that in the above example we used **onChange** which defined the desired output once we clicked the button. Several types of events exist within GEE, and a set of examples are given in the GEE [events](https://developers.google.com/earth-engine/guides/ui_events) page.

### Panels and Layouts

**Panels (ui.Panel)** are the containers where widgets are placed. Panels can bee added using **add()** or **remove()**. As panels are containers which hold the widgets, panels cannot be printed. The instance of root, **ui.root** holds all of the other panels and by default it contains an instance of the Map, **ui.Map**. To empty the canvas, simply use clear on the ui.root, for example : **ui.root.clear()**. Let's see an example :

Below we use several of the concepts to construct an app with widgets, panels and layouts which displays the bathymetry of the shallow regions of the Maldives (albeit using a low resolution version). To make it intreactive the app allows users to click at any point of the Map to get the bathymetry. Further, to illustrate the concept of using shapefiles and rasters we upload shapefiles for the lagoons as well as the islands and clip the batymetry raster to the lagoons and add the islands as a seperate layer.

So before we proceed we need to upload the shapefiles and geotiff. 

To do this simply go to **Assets > NEW ** , now select shapefiles for shapefiles and tiff for geotiff. Note that these are all geolocated so you will have to make sure the projection matches. You could add properties as well, for now we will just upload the files.

~~~javascript
var image = ee.Image("users/zubba1989/TUTORIALS/Surface5s");

Map.setOptions('SATELLITE');
Map.style().set('cursor', 'crosshair');

Map.centerObject(image,10)

//////////////////

var legend3 = ui.Panel({
style: {
position: 'top-center',
padding: '8px 15px',
fontWeight: 'bold',
fontSize: '18px',
textAlign : 'center'
}
});

// Create legend title
var legendTitle2 = ui.Label({
value: 'Click to get the bathymetry at the location.',
style: {
fontWeight: 'normal',
fontSize: '13px',
margin: '0 0 4px 0',
padding: '8px 15px',
textAlign : 'center'
}
});

var title = 'Bathymetry of the shallow regions of the Maldives (Low Resolution 5 arc seconds)';
legend3.widgets().set(0, ui.Label(title));

Map.add(legend3);

///////////////////

var image2 = ee.FeatureCollection("users/zubba1989/TUTORIALS/Islands_Maldives_Contours");
var image3 = ee.FeatureCollection('users/zubba1989/TUTORIALS/Lagoons_Maldives');

image = image.clip(image3)

Map.addLayer(image2.draw({color: '006600', strokeWidth: 1}), {}, 'Land');

var tmax = image.select('b1');

// Get a palette: a list of hex strings
var palettes = require('users/gena/packages:palettes');
var palette = palettes.kovesi.linear_blue_5_95_c73[7];

//var palette = palettes.matplotlib.viridis[7]

var elevationVis = {
  min: -250,
  max: 0.0,
  palette: ['011de2', 'afafaf', '3603ff', 'fff477', 'b42109'],
};

//Map.addLayer(image, elevationVis, 'Bathymetry @ 5 arcseconds (Diverged colour scheme)');
 
// Display max temp with defined palette stretched between selected min and max
Map.addLayer(tmax, {min: -15, max: 0, palette: palette}, 'Bathymetry @ 5 arcseconds');

Map.addLayer(image2.draw({color: '006600', strokeWidth: 1}), {}, 'Land');

//////////////////////////////////

// set position of panel

var viz = {
  min: -15,
  max: 0.0,
  palette: palette,
};


var legend = ui.Panel({
style: {
position: 'bottom-left',
padding: '8px 15px'
}
});

// Create legend title
var legendTitle = ui.Label({
value: 'Bathymetry (m)',
style: {
fontWeight: 'bold',
fontSize: '18px',
margin: '0 0 4px 0',
padding: '0'
}
});


// Add the title to the panel
legend.add(legendTitle);

// create the legend image
var lon = ee.Image.pixelLonLat().select('latitude');
var gradient = lon.multiply((viz.max-viz.min)/100.0).add(viz.min);
var legendImage = gradient.visualize(viz);

// create text on top of legend
var panel = ui.Panel({
widgets: [
ui.Label(viz['max'])
],
});

legend.add(panel);

// create thumbnail from the image
var thumbnail = ui.Thumbnail({
image: legendImage,
params: {bbox:'0,0,10,100', dimensions:'25x200'},
style: {padding: '1px', position: 'bottom-center'}
});

// add the thumbnail to the legend
legend.add(thumbnail);

// create text on top of legend
var panel = ui.Panel({
widgets: [////////////////////
ui.Label(viz['min'])
],
});

legend.add(panel);
Map.add(legend);

legend.add(legendTitle2);

////////////////////////////////

var legend2 = ui.Panel({
style: {
position: 'bottom-right',
padding: '8px 15px'
}
});

Map.onClick(function(coords) {
  var location = 'lon: ' + coords.lon.toFixed(4) + ' ' +
                 'lat: ' + coords.lat.toFixed(4);
  var click_point = ee.Geometry.Point(coords.lon, coords.lat);
  
  var demValue = image.reduceRegion(ee.Reducer.first(), click_point, 90).evaluate(function(val){
    var demText = 'Bathymetry Value: ' + val.b1;
    //legend2.widgets().set(0, ui.Label('aaaaaaaaaaaaaaaaaa'));
    legend2.widgets().set(0, ui.Label(demText));
  });
});

Map.add(legend2);

~~~

If all goes well, you will have a display like below :

![alt text for screen readers](bathy.png "Bathy App")


### Building an App

The above example is the skeleton for a GEE App. Lets convert the code we created above to publish our application. 

To do this simply go the **Apps** tab in your console, and then click **New App**. This will prompt a screen which requires you to give a name to your application. Next, for the Google Cloud Project, simply put : ee-username where username is your GEE username. You have the option to select the source code of your app at this stage. We will just select *Current contents of editor*. You also have the option to give a custom logo as well. Now click **publish** and you have almost have your app ready. 

We just have one step at this point. Remember that we added assets (the tiff and the shapefiles) and used those assets in building our apps. These assets must at this stage be made available to the application. To do this, navigate to each of the assets we uploaded, and click on the app. Now select **SHARE** and chose *Select an app*, from the dropdown select your application. 

Now, if you select **Apps** tab in the console, your app should appear with a link. Simply click on the link and you should be directed to your standalone GEE app. (Give a while for the app to initialize.)

### Exercise

1. Convert the area calculation app from Day 1 to an app. Below is the code we wrote :


```javascript

// Define the geometry

var selected_island = 
    /* color: #98ff00 */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[72.874231001045, 2.6781043350070313],
          [72.874231001045, 2.663528967877705],
          [72.8998943768995, 2.663528967877705],
          [72.8998943768995, 2.6781043350070313]]], null, false);

// Load the Sentinel 2 dataset and set the bounds to the selected island. 

var S1 = ee.ImageCollection('COPERNICUS/S2').filterBounds(selected_island).filterDate('2013-01-01','2020-09-01');

// Select images which are less than 10 percent cloudy

var filtered = S1.filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 10);

// Define a function to mask out clouds (https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2)

function maskS2clouds(image) {
    var qa = image.select('QA60');
  // Both flags should be set to zero, indicating clear conditions.
    var cloudBitMask = ee.Number(2).pow(10).int();
    var cirrusBitMask = ee.Number(2).pow(11).int();
    var mask = qa.bitwiseAnd(cloudBitMask).eq(0).and(qa.bitwiseAnd(cirrusBitMask).eq(0));
  return image.updateMask(mask);
}

// Define a function which calculates Non Descriptive Water Index and adds it as a band to each image of the dataset.
// NDWI is very sensitive to water, and is normally used to calculate properties of water bands. Here, we make use of the excellent sensitivity of the index to water and reverse it to get land instead. 

var calcNdwi = function(img) {
    var cloud_less = maskS2clouds(img)
    var ndwi = cloud_less.normalizedDifference(['B3','B8']).rename('NDWI')
  return img.addBands(ndwi)
}

// Next we map the functions and select the NDWI band 

S1 = filtered.map(calcNdwi).select('NDWI')

// Define a function which identifies land 

var clasifyNDWI = function(img){
    var ndwi = img.select('NDWI')
    var land = ndwi.lt(0.1).rename('Land')
    land = land.updateMask(land)
  return img.addBands(land)
}

// Next we use the map function to map the masking function across the filtered imagecollection

S1 = S1.map(clasifyNDWI)

// We want the output as a beautiful chart, note that we use a redducer to sum all of the pixels we associated with land        

var ClassChart = ui.Chart.image.series({
  imageCollection: S1.select('Land'),
  region: selected_island,
  reducer: ee.Reducer.sum(),
  scale: 100,
})
.setOptions({
    title: 'Island Size over time',
    fontSize: 12.5,
    hAxis: {'title': 'Date'},
    vAxis: {'title': 'Island size in Hecatars'},
    lineWidth: 0.15,
    pointSize: 2.0,
    color: '00FF00'
  })
  .setChartType('ScatterChart');
  
print (ClassChart)
```

2. Create an application which plots the Non Descrptive Vegetation Index (NDVI) when a user clicks a location. 

#### Hints

1. Check out the [code](https://github.com/zubba02/Island-Health-Explorer-Project/blob/main/Area_Veg_Explorer.js) on github for the GEE app which caluclates, the area and NDVI for all islands of the Maldives. 

*Note that, NDVI derived from landast is also available as a dataset on GEE , so you could just exploit it as well instead. For example check the official [Panels and Layouts](https://developers.google.com/earth-engine/guides/ui_panels#absolute) page on the GEE website.*

For those interested, it is absolutely worth checking out the python package (geemap)[https://geemap.org/], by (Qiusheng Wu)[https://github.com/giswqs]. He also has a (youtube)[https://www.youtube.com/channel/UC6V9oXSB3T1XSBsbqtYvKMg] channel with loads of stuff. 